# Exploring ML Fairness Concepts on the Titanic Dataset

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import seaborn as sns
import pandas as pd

%matplotlib notebook

In [ ]:
titanic_df = sns.load_dataset('titanic')

In [ ]:
import pandas_profiling
# titanic_df.profile_report()

In [ ]:
from titanic_pipeline import preprocess_data, create_classifier, evaluate_model

X_train, X_test, y_train, y_test = preprocess_data(titanic_df)

clf = create_classifier()
clf.fit(X_train, y_train)
evaluate_model(X_test, y_test, clf)

## Evaluating Male Population

In [ ]:
male_ind = X_test["sex"] == "male"
evaluate_model(X_test.loc[male_ind], y_test[male_ind], clf)

## Evaluating Female Population

In [ ]:
evaluate_model(X_test.loc[~male_ind], y_test[~male_ind], clf)

# Observational Metrics

Here we want to look at the observational fairness metrics, namely

### Equalised Odds

For all $a, a'$ and all $y$ we have 
$$P(\hat{Y} = y | A = a, Y = y) = P(\hat{Y} = y | A = a', Y = y)$$

> Translation: TP rates and TN rates are both equal over sensitive groups

### Calibration

For all $y$ and $a, a'$ we have
$$P(Y = y | A = a, \hat{Y} = y) = P(Y = y | A = a', \hat{Y} = y)$$

> We want the accuracy of the prediction (both positive/negative) to be equal over sensitive groups

### Demographic Parity/Disparate Impact

For all $y, a, a'$ we have
$$ P(\hat{Y} = y | A = a) = P(\hat{Y} = y | A = a')$$

> We want the distribution of predicted outcomes to be equal over sensitive groups

### Individual Fairness
We'll skip this due to difficulty choosing a proper metric for judging the similarity of two individuals.

> We want similar individuals from different groups to be treated equally.


In [ ]:
from fairness_metrics import eqaualised_odds, calibration, demographic_parity
        
eqaualised_odds(X_test, y_test, clf, "sex")
# TODO: Plot graph for different thresholds

In [ ]:
calibration(X_test, y_test, clf, "sex")

In [ ]:
demographic_parity(X_test, clf, "sex")

## Logistic Regression

In [ ]:
import statsmodels.formula.api as sm
from titanic_manual import Evaluate, ManualPreprocess, ConsistentIndex, CreateInput

# Load dataset
titanic_df = sns.load_dataset('titanic')
X_train, y_train, X_test, y_test = ManualPreprocess(titanic_df)


mod = sm.Logit(y_train, X_train)
res = mod.fit()
# print(res.summary()) # show results
y_test_pred = res.predict(X_test)

Evaluate(y_test, y_test_pred)
